In [1]:
import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds
import numpy as np

# tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

## Processing Data

In [4]:
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<UNK>'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

# paddings
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [116]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [117]:
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

## Model

In [118]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=6, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [119]:
num_epoch = 10
model.fit(padded,
         training_labels_final,
         epochs=num_epoch,
         validation_data=(testing_padded, testing_labels_final)
         )

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 9s 378us/sample - loss: 0.4496 - accuracy: 0.7776 - val_loss: 0.3274 - val_accuracy: 0.8544
Epoch 2/10
25000/25000 [==============================] - 8s 332us/sample - loss: 0.2080 - accuracy: 0.9219 - val_loss: 0.3395 - val_accuracy: 0.8538
Epoch 3/10
25000/25000 [==============================] - 9s 344us/sample - loss: 0.0796 - accuracy: 0.9809 - val_loss: 0.4222 - val_accuracy: 0.8412
Epoch 4/10
25000/25000 [==============================] - 9s 343us/sample - loss: 0.0205 - accuracy: 0.9976 - val_loss: 0.4865 - val_accuracy: 0.8430
Epoch 5/10
25000/25000 [==============================] - 9s 346us/sample - loss: 0.0057 - accuracy: 0.9996 - val_loss: 0.5464 - val_accuracy: 0.8408
Epoch 6/10
25000/25000 [==============================] - 9s 345us/sample - loss: 0.0021 - accuracy: 0.9999 - val_loss: 0.5918 - val_accuracy: 0.8426
Epoch 7/10
25000/25000 [==========================

## Explore
Save result as TSV and visualize with TF Projector

In [133]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

(10000, 16)


In [132]:
review = 'It was an amazing movie'
seq = tokenizer.texts_to_sequences(review)
padded = pad_sequences(seq, maxlen=max_length)


model.predict(
    padded
)

array([[0.41226166],
       [0.0994468 ],
       [0.35477346],
       [0.30235693],
       [0.38272068],
       [0.28558344],
       [0.35477346],
       [0.38272068],
       [0.5083949 ],
       [0.35477322],
       [0.38272068],
       [0.3786134 ],
       [0.38272068],
       [0.12643465],
       [0.41226166],
       [0.50839525],
       [0.50807124],
       [0.35477346],
       [0.3786134 ],
       [0.43026483],
       [0.09257936],
       [0.41226166],
       [0.47910312]], dtype=float32)

In [6]:
training_labels_final

array([0, 0, 0, ..., 0, 0, 1])